# Pipeline Engine — Interactive Demo

This notebook walks through every feature of the generic pipeline engine.
Each cell is self-contained — run them top to bottom.

**No external dependencies** — only the `pipeline/` package.

## Setup & Imports

In [1]:
import sys, time
from types import MappingProxyType
from pathlib import Path

# Jupyter already runs an asyncio event loop.  Pipeline.run() calls
# asyncio.run() internally, which would fail.  nest_asyncio patches the
# loop to allow nested calls.
import nest_asyncio
nest_asyncio.apply()

# Walk up from the notebook directory until we find the project root
# (identified by containing a `pipeline/` package directory).
_here = Path.cwd()
_root = _here
for _p in [_here] + list(_here.parents):
    if (_p / "pipeline" / "__init__.py").exists():
        _root = _p
        break
sys.path.insert(0, str(_root))

# Clear any stale import (ace/pipeline can shadow the top-level pipeline/).
if "pipeline" in sys.modules:
    del sys.modules["pipeline"]

from pipeline import (
    Pipeline,
    Branch,
    MergeStrategy,
    StepContext,
    SampleResult,
    PipelineOrderError,
    BranchError,
)

def show(results: list[SampleResult]) -> None:
    """Pretty-print a list of SampleResult."""
    for r in results:
        tag = "OK" if r.error is None else f"FAIL @ {r.failed_at}"
        print(f"  [{tag}] sample={r.sample!r}")
        if r.output:
            named = {k: getattr(r.output, k) for k in ("agent_output", "environment_result", "reflection") if getattr(r.output, k) is not None}
            if named:
                print(f"         fields:   {named}")
            meta = dict(r.output.metadata)
            if meta:
                print(f"         metadata: {meta}")
        if r.error:
            print(f"         error:    {r.error}")

print(f"Project root: {_root}")
print(f"pipeline pkg: {Pipeline.__module__}")

Project root: /home/david/Desktop/projects/Kayba/agentic-context-engine
pipeline pkg: pipeline.pipeline


## Step Definitions

A **step** is any object with:
- `requires: frozenset[str]` — metadata keys it reads
- `provides: frozenset[str]` — metadata keys it writes
- `__call__(ctx: StepContext) -> StepContext`

No base class — pure duck typing via `StepProtocol`.

In [2]:
class Tokenize:
    """Split sample text into words, store token list and count."""
    requires = frozenset()
    provides = frozenset({"tokens", "word_count"})

    def __call__(self, ctx: StepContext) -> StepContext:
        tokens = str(ctx.sample).split()
        print(f"    [Tokenize]  '{ctx.sample}' → {len(tokens)} tokens")
        return ctx.replace(metadata=MappingProxyType({
            **ctx.metadata, "tokens": tokens, "word_count": len(tokens),
        }))


class Uppercase:
    """Uppercase each token. Requires 'tokens' in metadata."""
    requires = frozenset({"tokens"})
    provides = frozenset({"upper_tokens"})

    def __call__(self, ctx: StepContext) -> StepContext:
        upper = [t.upper() for t in ctx.metadata["tokens"]]
        print(f"    [Uppercase] {ctx.metadata['tokens']} → {upper}")
        return ctx.replace(metadata=MappingProxyType({**ctx.metadata, "upper_tokens": upper}))


class Reverse:
    """Reverse each token. Designed to run in parallel with Uppercase."""
    requires = frozenset({"tokens"})
    provides = frozenset({"reversed_tokens"})

    def __call__(self, ctx: StepContext) -> StepContext:
        rev = [t[::-1] for t in ctx.metadata["tokens"]]
        print(f"    [Reverse]   {ctx.metadata['tokens']} → {rev}")
        return ctx.replace(metadata=MappingProxyType({**ctx.metadata, "reversed_tokens": rev}))


class Summarize:
    """Combine processed metadata into a final agent_output string."""
    requires = frozenset({"upper_tokens", "reversed_tokens", "word_count"})
    provides = frozenset({"agent_output"})

    def __call__(self, ctx: StepContext) -> StepContext:
        summary = (
            f"{ctx.metadata['word_count']} words | "
            f"upper={ctx.metadata['upper_tokens']} | "
            f"rev={ctx.metadata['reversed_tokens']}"
        )
        print(f"    [Summarize] → {summary}")
        return ctx.replace(agent_output=summary)


class Boom:
    """Always fails — used to demonstrate error handling."""
    requires = frozenset()
    provides = frozenset()

    def __call__(self, ctx: StepContext) -> StepContext:
        raise RuntimeError(f"Boom on sample={ctx.sample!r}!")

---
## 1. Basic Linear Pipeline

Chain steps with `.then()`. The pipeline infers its **contracts**
(`requires` / `provides`) from the step chain automatically.

In [22]:
pipe = Pipeline().then(Tokenize()).then(Uppercase())

print("Pipeline contracts:")
print(f"  requires = {pipe.requires}   ← external inputs the caller must provide")
print(f"  provides = {pipe.provides}   ← everything the pipeline writes")
print()

results = pipe.run(["hello world", "pipeline engine demo"])
show(results)

Pipeline contracts:
  requires = frozenset()   ← external inputs the caller must provide
  provides = frozenset({'upper_tokens', 'tokens', 'word_count'})   ← everything the pipeline writes

    [Tokenize]  'hello world' → 2 tokens
    [Uppercase] ['hello', 'world'] → ['HELLO', 'WORLD']
    [Tokenize]  'pipeline engine demo' → 3 tokens
    [Uppercase] ['pipeline', 'engine', 'demo'] → ['PIPELINE', 'ENGINE', 'DEMO']
  [OK] sample='hello world'
         metadata: {'tokens': ['hello', 'world'], 'word_count': 2, 'upper_tokens': ['HELLO', 'WORLD']}
  [OK] sample='pipeline engine demo'
         metadata: {'tokens': ['pipeline', 'engine', 'demo'], 'word_count': 3, 'upper_tokens': ['PIPELINE', 'ENGINE', 'DEMO']}


---
## 2. Contract Validation

The engine validates step ordering at **construction time**.
If a step needs a field that a *later* step provides → `PipelineOrderError`.

Fields not provided by *any* step are treated as **external inputs** — no error.

In [4]:
# Wrong order: Uppercase needs 'tokens', but Tokenize comes after
print("Trying: Pipeline().then(Uppercase()).then(Tokenize())\n")

try:
    Pipeline().then(Uppercase()).then(Tokenize())
except PipelineOrderError as e:
    print(f"  Caught PipelineOrderError:\n  {e}\n")

# External input: 'tokens' not produced by anyone → valid, caller must provide it
p = Pipeline().then(Uppercase())
print(f"External input is OK:  requires = {p.requires}")

Trying: Pipeline().then(Uppercase()).then(Tokenize())

  Caught PipelineOrderError:
  Uppercase requires {'tokens'} but these are produced by a later step — check step ordering.

External input is OK:  requires = frozenset({'tokens'})


---
## 3. Branch — Parallel Fork/Join

`.branch()` fans out to N child pipelines in parallel, then **merges**
their outputs back into a single `StepContext`.

```
                 ┌── Uppercase ──┐
  Tokenize ──►──┤               ├──► Summarize
                 └── Reverse  ──┘
```

In [5]:
pipe = (
    Pipeline()
    .then(Tokenize())
    .branch(
        Pipeline().then(Uppercase()),
        Pipeline().then(Reverse()),
        merge=MergeStrategy.RAISE_ON_CONFLICT,
    )
    .then(Summarize())
)

print(f"requires = {pipe.requires}")
print(f"provides = {pipe.provides}\n")

results = pipe.run(["fork join"])
show(results)

requires = frozenset()
provides = frozenset({'upper_tokens', 'reversed_tokens', 'word_count', 'agent_output', 'tokens'})

    [Tokenize]  'fork join' → 2 tokens
    [Uppercase] ['fork', 'join'] → ['FORK', 'JOIN']
    [Reverse]   ['fork', 'join'] → ['krof', 'nioj']
    [Summarize] → 2 words | upper=['FORK', 'JOIN'] | rev=['krof', 'nioj']
  [OK] sample='fork join'
         fields:   {'agent_output': "2 words | upper=['FORK', 'JOIN'] | rev=['krof', 'nioj']"}
         metadata: {'tokens': ['fork', 'join'], 'word_count': 2, 'upper_tokens': ['FORK', 'JOIN'], 'reversed_tokens': ['krof', 'nioj']}


---
## 4. Merge Strategies

When branches write the **same named field**, the merge strategy decides what happens:

| Strategy | Behaviour |
|---|---|
| `RAISE_ON_CONFLICT` | `ValueError` if any named field differs (metadata always LWW) |
| `LAST_WRITE_WINS` | Last branch's value wins for every field |
| `NAMESPACED` | Each branch stored at `metadata["branch_N"]`, no conflict possible |

In [6]:
class WriteAnswer:
    requires = frozenset()
    provides = frozenset({"agent_output"})
    def __init__(self, val: str):
        self.val = val
    def __call__(self, ctx):
        return ctx.replace(agent_output=self.val)

ctx = StepContext(sample="q")

In [7]:
# a) RAISE_ON_CONFLICT — two branches write different values → error
print("a) RAISE_ON_CONFLICT with conflict:\n")

b = Branch(
    Pipeline().then(WriteAnswer("yes")),
    Pipeline().then(WriteAnswer("no")),
    merge=MergeStrategy.RAISE_ON_CONFLICT,
)
try:
    b(ctx)
except ValueError as e:
    print(f"   Caught ValueError: {e}")

a) RAISE_ON_CONFLICT with conflict:

   Caught ValueError: Branch outputs conflict on fields {'agent_output'}. Use a different merge strategy or ensure branches write disjoint fields.


In [14]:
# b) LAST_WRITE_WINS — second branch always wins
print("b) LAST_WRITE_WINS:\n")

b = Branch(
    Pipeline().then(WriteAnswer("yes")),
    Pipeline().then(WriteAnswer("no")),
    merge=MergeStrategy.LAST_WRITE_WINS,
)
out = b(ctx)
print(f"   agent_output = {out.agent_output!r}   ← second branch wins")

b) LAST_WRITE_WINS:

   agent_output = 'no'   ← second branch wins


In [15]:
# c) NAMESPACED — each branch isolated, accessible via metadata key
print("c) NAMESPACED:\n")

b = Branch(
    Pipeline().then(WriteAnswer("yes")),
    Pipeline().then(WriteAnswer("no")),
    merge=MergeStrategy.NAMESPACED,
)
out = b(ctx)
print(f"   agent_output          = {out.agent_output!r}   ← from first branch")
print(f"   branch_0.agent_output = {out.metadata['branch_0'].agent_output!r}")
print(f"   branch_1.agent_output = {out.metadata['branch_1'].agent_output!r}")

c) NAMESPACED:

   agent_output          = 'yes'   ← from first branch
   branch_0.agent_output = 'yes'
   branch_1.agent_output = 'no'


---
## 5. Error Handling

Every sample produces a `SampleResult` — nothing is dropped silently.
A failing sample sets `error` and `failed_at`; other samples continue.

In [16]:
print("All samples fail:\n")
results = Pipeline().then(Tokenize()).then(Boom()).run(["good luck"])
show(results)

All samples fail:

    [Tokenize]  'good luck' → 2 tokens
  [FAIL @ Boom] sample='good luck'
         error:    Boom on sample='good luck'!


In [17]:
print("Mixed success / failure:\n")

class MaybeBoom:
    requires = frozenset()
    provides = frozenset()
    def __call__(self, ctx):
        if "bad" in str(ctx.sample):
            raise RuntimeError("bad sample!")
        return ctx

results = Pipeline().then(Tokenize()).then(MaybeBoom()).run(["ok", "bad input", "fine"])
show(results)

Mixed success / failure:

    [Tokenize]  'ok' → 1 tokens
    [Tokenize]  'bad input' → 2 tokens
    [Tokenize]  'fine' → 1 tokens
  [OK] sample='ok'
         metadata: {'tokens': ['ok'], 'word_count': 1}
  [FAIL @ MaybeBoom] sample='bad input'
         error:    bad sample!
  [OK] sample='fine'
         metadata: {'tokens': ['fine'], 'word_count': 1}


---
## 6. Async Boundary — Fire-and-Forget Background

Set `async_boundary = True` on a step. Everything from that step onward
runs in a **background thread**. `run()` returns immediately.

```
  Foreground (fast)         Background (slow)
  ┌──────────┐             ┌───────────┐
  │ Tokenize │ ──────►──── │ SlowScore │
  └──────────┘             └───────────┘
       │                        │
    run() returns          updated later
```

Call `wait_for_background()` to join all background threads.

In [18]:
class SlowScore:
    """Expensive scoring step that runs in background."""
    requires = frozenset()
    provides = frozenset({"score"})
    async_boundary = True
    max_workers = 2

    def __call__(self, ctx: StepContext) -> StepContext:
        time.sleep(0.1)
        score = ctx.metadata.get("word_count", 0) * 10
        print(f"    [SlowScore] sample={ctx.sample!r} score={score}  (background)")
        return ctx.replace(metadata=MappingProxyType({**ctx.metadata, "score": score}))

pipe = Pipeline().then(Tokenize()).then(SlowScore())

t0 = time.monotonic()
results = pipe.run(["fast return", "also fast"], workers=2)
elapsed = time.monotonic() - t0

print(f"\nrun() returned in {elapsed:.3f}s — background still scoring\n")
show(results)

    [Tokenize]  'fast return' → 2 tokens
    [Tokenize]  'also fast' → 2 tokens

run() returned in 0.004s — background still scoring

  [OK] sample='fast return'
  [OK] sample='also fast'


    [SlowScore] sample='fast return' score=20  (background)
    [SlowScore] sample='also fast' score=20  (background)


In [19]:
# Now wait for background to finish and inspect the updated results
print("Waiting for background...\n")
pipe.wait_for_background(timeout=5.0)
print("Done!\n")
show(results)

Waiting for background...

Done!

  [OK] sample='fast return'
         metadata: {'tokens': ['fast', 'return'], 'word_count': 2, 'score': 20}
  [OK] sample='also fast'
         metadata: {'tokens': ['also', 'fast'], 'word_count': 2, 'score': 20}


---
## 7. Nested Pipelines

A `Pipeline` satisfies `StepProtocol`, so it can be used as a step
inside another pipeline. Contracts are inferred recursively.

In [20]:
inner = Pipeline().then(Tokenize()).then(Uppercase())
outer = Pipeline().then(inner).then(Reverse())

print(f"Inner: requires={inner.requires}, provides={inner.provides}")
print(f"Outer: requires={outer.requires}, provides={outer.provides}\n")

results = outer.run(["nested demo"])
show(results)

Inner: requires=frozenset(), provides=frozenset({'upper_tokens', 'tokens', 'word_count'})
Outer: requires=frozenset(), provides=frozenset({'upper_tokens', 'tokens', 'word_count', 'reversed_tokens'})

    [Tokenize]  'nested demo' → 2 tokens
    [Uppercase] ['nested', 'demo'] → ['NESTED', 'DEMO']
    [Reverse]   ['nested', 'demo'] → ['detsen', 'omed']
  [OK] sample='nested demo'
         metadata: {'tokens': ['nested', 'demo'], 'word_count': 2, 'upper_tokens': ['NESTED', 'DEMO'], 'reversed_tokens': ['detsen', 'omed']}


---
## 8. Workers — Concurrent Sample Processing

The `workers` parameter on `run()` controls how many samples are processed
in parallel (via an `asyncio.Semaphore` in the foreground event loop).
This is independent of `max_workers` on individual steps.

In [21]:
class SlowStep:
    requires = frozenset()
    provides = frozenset({"done"})
    def __call__(self, ctx):
        time.sleep(0.1)
        return ctx.replace(metadata=MappingProxyType({**ctx.metadata, "done": True}))

samples = [f"s{i}" for i in range(6)]
pipe = Pipeline().then(SlowStep())

t0 = time.monotonic()
pipe.run(samples, workers=1)
seq = time.monotonic() - t0

t0 = time.monotonic()
pipe.run(samples, workers=6)
par = time.monotonic() - t0

print(f"  workers=1 : {seq:.2f}s")
print(f"  workers=6 : {par:.2f}s")
print(f"  speedup   : {seq / par:.1f}x")

  workers=1 : 0.60s
  workers=6 : 0.10s
  speedup   : 5.9x


---
## Summary

| Concept | API |
|---|---|
| Linear chain | `Pipeline().then(A()).then(B())` |
| Contract inference | `pipe.requires`, `pipe.provides` |
| Parallel fork/join | `.branch(Pipeline().then(A()), Pipeline().then(B()))` |
| Merge control | `merge=MergeStrategy.RAISE_ON_CONFLICT / LAST_WRITE_WINS / NAMESPACED` |
| Error isolation | `SampleResult.error`, `SampleResult.failed_at` |
| Background execution | `async_boundary = True` on a step class |
| Background join | `pipe.wait_for_background(timeout=...)` |
| Nesting | Use a `Pipeline` as a step inside another `Pipeline` |
| Sample concurrency | `pipe.run(samples, workers=N)` |

The pipeline engine is **domain-agnostic** — it knows nothing about ACE.
The `ace2/` package will add domain-specific steps (Agent, Evaluate,
Reflect, Update) on top of this engine.